<a href="https://colab.research.google.com/github/Ruby-L-i-n/CSDS234_FinalProject/blob/main/FeatureGeneration%2Binput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install textblob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import re

# Reading the Track Information Details

In [5]:
track_df_1 = pd.read_csv("/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/tracks_info_0-34443.csv")

track_df_1

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Lose Control (feat. Ciara & Fat Man Scoop),Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,"['dance pop', 'hip hop', 'hip pop', 'neo soul'...",69,0.904,0.813,4,-7.105,0,...,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864,4
1,Toxic,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"['dance pop', 'pop']",79,0.774,0.838,5,-3.914,0,...,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800,4
2,Crazy In Love,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,"['pop', 'r&b']",84,0.664,0.758,2,-6.583,0,...,0.0598,0.701,99.259,audio_features,0WqIKmW4BTrj3eJFmnCKMv,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933,4
3,Rock Your Body,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,"['dance pop', 'pop']",79,0.892,0.714,4,-6.055,0,...,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,spotify:track:1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267,4
4,It Wasn't Me,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,"['pop rap', 'reggae fusion']",70,0.853,0.606,0,-4.596,1,...,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,spotify:track:1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34438,I Don't Know,Jon D,spotify:artist:5HCypjplgh5uQezvBpOfXN,['channel pop'],39,0.669,0.228,2,-12.119,1,...,0.0944,0.402,83.024,audio_features,3uCHI1gfOUL5j5swEh0TcH,spotify:track:3uCHI1gfOUL5j5swEh0TcH,https://api.spotify.com/v1/tracks/3uCHI1gfOUL5...,https://api.spotify.com/v1/audio-analysis/3uCH...,189184,4
34439,The Answer,Big Words,spotify:artist:0sHN89qak07mnug3LVVjzP,['australian r&b'],34,0.493,0.727,1,-5.031,1,...,0.1290,0.289,73.259,audio_features,0P1oO2gREMYUCoOkzYAyFu,spotify:track:0P1oO2gREMYUCoOkzYAyFu,https://api.spotify.com/v1/tracks/0P1oO2gREMYU...,https://api.spotify.com/v1/audio-analysis/0P1o...,263680,4
34440,25.22,Allan Rayman,spotify:artist:6Yv6OBXD6ZQakEljaGaDAk,['canadian contemporary r&b'],48,0.702,0.524,7,-10.710,1,...,0.2980,0.265,140.089,audio_features,2oM4BuruDnEvk59IvIXCwn,spotify:track:2oM4BuruDnEvk59IvIXCwn,https://api.spotify.com/v1/tracks/2oM4BuruDnEv...,https://api.spotify.com/v1/audio-analysis/2oM4...,189213,4
34441,Good Feeling,Jon Jason,spotify:artist:77bNdkKYBBmc30CisCA6tE,[],2,0.509,0.286,8,-14.722,1,...,0.1310,0.259,121.633,audio_features,4Ri5TTUgjM96tbQZd5Ua7V,spotify:track:4Ri5TTUgjM96tbQZd5Ua7V,https://api.spotify.com/v1/tracks/4Ri5TTUgjM96...,https://api.spotify.com/v1/audio-analysis/4Ri5...,194720,4


In [54]:
track_df2 = pd.read_csv('/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/track_info_input_t1.csv')
# generating playlist recommendations

track_df2
track_df = pd.concat([track_df_1,track_df2],ignore_index=True)
track_df.drop_duplicates(inplace = True)
track_df.reset_index(drop=True, inplace = True)


# Analysis of Track Name using Sentiment Analysis
Subjectivity [0:1]: 0 meaning factual, whereas 1 means opinionated.
Polarity [-1,1]: -1 is negative connotation, 1 means positive connotation

In [55]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

def getAnalysis(score, task="polarity"):
  if task == "subjectivity":
    if score < 1/3:
      return "low"
    elif score > 1/3:
      return "high"
    else:
      return "medium"
  else:
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

def sentiment_analysis(df, text_col):
  # Perform sentiment analysis on text
  df['subjectivity'] = df[text_col].apply(getSubjectivity).apply(lambda x: getAnalysis(x,"subjectivity"))
  df['polarity'] = df[text_col].apply(getPolarity).apply(getAnalysis)
  return df

In [56]:
# Show result
sentiment = sentiment_analysis(track_df, "track_name")

In [57]:
def ohe_process(df, column, new_name):
  tf_df = pd.get_dummies(df[column])
  feature_names = tf_df.columns
  tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
  tf_df.reset_index(drop = True, inplace = True)
  return tf_df

In [58]:
def subjectivity(df):
  subject_ohe = ohe_process(df, "subjectivity", "subjectivity") * 0.3
  return subject_ohe

subjectivity(track_df)

,subjectivity|high,subjectivity|low,subjectivity|medium
0,0.0,0.3,0.0
1,0.0,0.3,0.0
2,0.3,0.0,0.0
3,0.0,0.3,0.0
4,0.0,0.3,0.0
...,...,...,...
34510,0.3,0.0,0.0
34511,0.0,0.3,0.0
34512,0.0,0.3,0.0
34513,0.0,0.3,0.0


In [59]:
def polarity(df):
  polarity_ohe = ohe_process(df, "polarity", "polarity") * 0.5
  return polarity_ohe

# Analysis on Genres
Use Term Frequency-Inverse Document Frequency to determine the importance of each genre.

In [60]:
def genre_tfidf(df):
  tfidf = TfidfVectorizer()
  tfidf_track_matrix = tfidf.fit_transform(df['genres'])
  genre_df = pd.DataFrame(tfidf_track_matrix.toarray())
  genre_df.columns = ['genre' + '|' + i for i in tfidf.get_feature_names_out()]
  return genre_df

  # genre_df['genre|rap']

# Normalization on Floating Values
Analysis on popularity, key, mode, and other audio features

In [61]:
def popularity_normalization(df):
  popularity = df[['popularity']]
  scaler = MinMaxScaler()
  popularity_df = pd.DataFrame(scaler.fit_transform(popularity), columns = ['popularity'])
  return popularity_df

In [62]:
# audio features
def audio_features(df):
  #one-hot encoding
  key_ohe = ohe_process(df, "key", "key") * 0.5
  mode_ohe = ohe_process(df, "mode", "mode") * 0.5
  #normalization of other audio_features
  floats_cols = df.dtypes[df.dtypes == 'float64'].index.values #selects the audio feature colns
  floats = df[floats_cols].reset_index(drop = True)
  scaler = MinMaxScaler()
  floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2
  #putting them altogether
  audio_feats_df = pd.concat([key_ohe, mode_ohe, floats_scaled], axis=1)
  return audio_feats_df

audio_features(track_df)


,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.162146,0.1202,0.167052,0.053015,0.015602,0.000006,0.02900,0.148297,0.145893
34511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.155466,0.1428,0.174904,0.008337,0.070683,0.000000,0.03000,0.168737,0.119877
34512,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,...,0.0,0.105263,0.1462,0.174177,0.011580,0.068675,0.000203,0.06220,0.132665,0.158625
34513,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.142510,0.1594,0.172300,0.009875,0.016586,0.000150,0.01092,0.165331,0.127675


# Generating the Feature Set

In [63]:
def generate_feature_set(df):
  popularity = popularity_normalization(df)
  genre = genre_tfidf(df)
  subjectivity_names = subjectivity(df)
  audio_features_df = audio_features(df)

  feature_set = pd.concat([genre, popularity, subjectivity_names, audio_features_df], axis = 1)
  return feature_set

generate_feature_set(track_df)

,genre|21st,genre|432hz,genre|abstract,genre|acid,genre|acousmatic,genre|acoustic,genre|action,genre|adoracion,genre|adult,genre|adventista,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.162146,0.1202,0.167052,0.053015,0.015602,0.000006,0.02900,0.148297,0.145893
34511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.155466,0.1428,0.174904,0.008337,0.070683,0.000000,0.03000,0.168737,0.119877
34512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.105263,0.1462,0.174177,0.011580,0.068675,0.000203,0.06220,0.132665,0.158625
34513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.142510,0.1594,0.172300,0.009875,0.016586,0.000150,0.01092,0.165331,0.127675


# Picking a playlist

In [64]:
# generating playlist recommendations

track_df1 = pd.read_csv('/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/track_info_input_t1.csv')
track_df1.head()

,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,exes,Tate McRae,spotify:artist:45dkTj5sMRSjrmBSBeiHym,"['alt z', 'pop']",84,0.838,0.569,5,-6.324,0,...,0.1020,0.551,136.965,audio_features,0HD8mbiPjp3o94X3EaZp0o,spotify:track:0HD8mbiPjp3o94X3EaZp0o,https://api.spotify.com/v1/tracks/0HD8mbiPjp3o...,https://api.spotify.com/v1/audio-analysis/0HD8...,159400,4
1,Lovin On Me,Jack Harlow,spotify:artist:2LIk90788K0zvyj2JJVwkJ,"['deep underground hip hop', 'kentucky hip hop...",83,0.943,0.558,2,-4.911,1,...,0.0937,0.606,104.983,audio_features,4xhsWYTOGcal8zt0J161CU,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4
2,Surround Sound (feat. 21 Savage & Baby Tate),JID,spotify:artist:6U3ybJ9UHNKEdsH7ktGBZ7,"['hip hop', 'pop rap', 'rap', 'underground hip...",76,0.575,0.560,5,-7.302,1,...,0.2570,0.471,75.956,audio_features,1udwFobQ1JoOdWPQrp2b6u,spotify:track:1udwFobQ1JoOdWPQrp2b6u,https://api.spotify.com/v1/tracks/1udwFobQ1JoO...,https://api.spotify.com/v1/audio-analysis/1udw...,229938,4
3,Water,Tyla,spotify:artist:3SozjO3Lat463tQICI9LcE,[],75,0.673,0.722,3,-3.495,0,...,0.1370,0.519,117.187,audio_features,5aIVCx5tnk0ntmdiinnYvw,spotify:track:5aIVCx5tnk0ntmdiinnYvw,https://api.spotify.com/v1/tracks/5aIVCx5tnk0n...,https://api.spotify.com/v1/audio-analysis/5aIV...,200256,4
4,Man of the Hour,Mackgee,spotify:artist:6CbJfzNqnSrBslhxbSaKPu,[],12,0.895,0.804,9,-5.587,0,...,0.0585,0.801,91.020,audio_features,0SXUtBuoJMQNab3zgkr8lw,spotify:track:0SXUtBuoJMQNab3zgkr8lw,https://api.spotify.com/v1/tracks/0SXUtBuoJMQN...,https://api.spotify.com/v1/audio-analysis/0SXU...,154628,4


In [65]:
playlist_df = pd.read_csv('/content/drive/Shareddrives/CSDS 234 Final Project/spotify_user_input_dataset/playlist_info_input_t1.csv')
playlist_df.head()

,pid,name,collaborative,modified_at,num_tracks,num_albums,num_followers,playlist_pid,track_pos,track_name,track_uri,artist_name,artist_uri,album_name,album_uri,duration_ms
0,6OcWuyMNKus06gmrh4FEiX,Popular TikTok Songs 2023,False,1702044940,75,70,8938,6OcWuyMNKus06gmrh4FEiX,0,exes,0HD8mbiPjp3o94X3EaZp0o,Tate McRae,spotify:artist:45dkTj5sMRSjrmBSBeiHym,exes,spotify:album:54FpKE2Pug3xbPz26WUVSH,159400
1,6OcWuyMNKus06gmrh4FEiX,Popular TikTok Songs 2023,False,1702044940,75,70,8938,6OcWuyMNKus06gmrh4FEiX,1,Lovin On Me,4xhsWYTOGcal8zt0J161CU,Jack Harlow,spotify:artist:2LIk90788K0zvyj2JJVwkJ,Lovin On Me,spotify:album:6VCO0fDBGbRW8mCEvV95af,138411
2,6OcWuyMNKus06gmrh4FEiX,Popular TikTok Songs 2023,False,1702044940,75,70,8938,6OcWuyMNKus06gmrh4FEiX,2,Surround Sound (feat. 21 Savage & Baby Tate),1udwFobQ1JoOdWPQrp2b6u,JID,spotify:artist:6U3ybJ9UHNKEdsH7ktGBZ7,The Forever Story (Extended Version),spotify:album:4rJDCELWL0fjdmN9Gn4f4g,229938
3,6OcWuyMNKus06gmrh4FEiX,Popular TikTok Songs 2023,False,1702044940,75,70,8938,6OcWuyMNKus06gmrh4FEiX,3,Water,5aIVCx5tnk0ntmdiinnYvw,Tyla,spotify:artist:3SozjO3Lat463tQICI9LcE,Water,spotify:album:22sXXkKgjEuawIFL1e1tRw,200255
4,6OcWuyMNKus06gmrh4FEiX,Popular TikTok Songs 2023,False,1702044940,75,70,8938,6OcWuyMNKus06gmrh4FEiX,4,Man of the Hour,0SXUtBuoJMQNab3zgkr8lw,Mackgee,spotify:artist:6CbJfzNqnSrBslhxbSaKPu,Man of the Hour,spotify:album:0RL00G9zEd1LUFknWPS9fs,154628


# Processing playlists

In [79]:
import numpy as np

#feature set of all tracks
total_tracks_features = generate_feature_set(track_df)
total_tracks_features['track_uri'] = track_df['uri']
total_tracks_features.drop_duplicates(inplace = True)
total_tracks_features

# feature set summary vector for a playlist
def input_playlist_features_vector (pid):
  playlist_uris = list(track_df1['uri'])
  input_playlist_features_df = total_tracks_features[total_tracks_features['track_uri'].isin(playlist_uris)]
  input_playlist_features_df = input_playlist_features_df.drop('track_uri', axis = 1)
  feature_vector = input_playlist_features_df.sum(axis=0)
  return feature_vector

input_playlist_features_vector(playlist_df.loc[0,'pid'])

genre|21st          0.000000
genre|432hz         0.000000
genre|abstract      0.000000
genre|acid          0.000000
genre|acousmatic    0.000000
                      ...   
acousticness        3.973719
instrumentalness    0.301557
liveness            2.536980
valence             8.199800
tempo               8.601684
Length: 1359, dtype: float64

In [82]:
def generate_nonplaylist_features(pid):
    playlist_uris = set(track_df1['uri'])

    # Filter non-playlist features
    nonplaylist_features = total_tracks_features[~total_tracks_features['track_uri'].isin(playlist_uris)]
    num_false_values = len(~total_tracks_features['track_uri'].isin(playlist_uris)) - sum(~total_tracks_features['track_uri'].isin(playlist_uris))
    nonplaylist_features = nonplaylist_features.drop('track_uri', axis = 1)
    nonplaylist_features.reset_index(drop = True, inplace = True)

    return nonplaylist_features

generate_nonplaylist_features(playlist_df.loc[0, 'pid'])


,genre|21st,genre|432hz,genre|abstract,genre|acid,genre|acousmatic,genre|acoustic,genre|action,genre|adoracion,genre|adult,genre|adventista,...,mode|1,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.182996,0.1626,0.168547,0.025156,0.006245,0.001401,0.00942,0.162325,0.114421
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.156680,0.1676,0.178715,0.023701,0.005000,0.005025,0.04840,0.185170,0.130453
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.134413,0.1516,0.170210,0.043659,0.000478,0.000000,0.01196,0.140481,0.090525
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.180567,0.1428,0.171892,0.029314,0.040361,0.000047,0.01042,0.163727,0.092087
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.172672,0.1212,0.176541,0.014823,0.011265,0.000000,0.06260,0.131062,0.086421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.135425,0.0456,0.152570,0.014345,0.159036,0.013065,0.01888,0.080561,0.075718
34435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.099798,0.1454,0.175155,0.045114,0.017530,0.000000,0.02580,0.057916,0.066813
34436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.142105,0.1048,0.157060,0.016486,0.066667,0.011116,0.05960,0.053106,0.127762
34437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.5,0.103036,0.0572,0.144276,0.025572,0.080723,0.000002,0.02620,0.051904,0.110930


# Implementing Recommendation System

In [84]:
def recommend_playlist(input_pid):
  input_playlist_features = input_playlist_features_vector(input_pid)
  nonplaylist_features = generate_nonplaylist_features(input_pid)

  playlist_uris = list(track_df1['uri'])
  nonplaylist_df = track_df[~track_df['uri'].isin(playlist_uris)]
  nonplaylist_df.reset_index(drop = True, inplace = True)
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]

  nonplaylist_df = nonplaylist_df.sort_values('similarity',ascending = False)
  return nonplaylist_df

recommend_playlist(playlist_df.loc[0,'pid'])

# (34442, 1360)
# (34442, 1359)
# (1359,)
# (34439, 23)

<ipython-input-84-5334da316217>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]


,track_name,artist_name,artist_uri,genres,popularity,danceability,energy,key,loudness,mode,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,subjectivity,polarity,similarity
1184,Coffee (F***ing),Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.550,0.661,1,-7.363,1,...,audio_features,60kqoWSXSbiDJaDxgi8GG1,spotify:track:60kqoWSXSbiDJaDxgi8GG1,https://api.spotify.com/v1/tracks/60kqoWSXSbiD...,https://api.spotify.com/v1/audio-analysis/60kq...,313907,4,low,Neutral,0.840720
24288,Shockandawe,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.766,0.902,0,-7.067,1,...,audio_features,11SY8jJkg1CtUgHx37qUlm,spotify:track:11SY8jJkg1CtUgHx37qUlm,https://api.spotify.com/v1/tracks/11SY8jJkg1Ct...,https://api.spotify.com/v1/audio-analysis/11SY...,150730,4,low,Neutral,0.840522
6888,Use Me,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.432,0.867,9,-6.809,1,...,audio_features,2dpZuh58EaAhfLxzAH7AxT,spotify:track:2dpZuh58EaAhfLxzAH7AxT,https://api.spotify.com/v1/tracks/2dpZuh58EaAh...,https://api.spotify.com/v1/audio-analysis/2dpZ...,280267,4,low,Neutral,0.839200
8171,All I Want Is You,Miguel,spotify:artist:360IAlyVv4PCEVjgyMZrxK,['r&b'],77,0.776,0.603,5,-5.696,1,...,audio_features,5VA4Ispp52EA1sOqzMz3Av,spotify:track:5VA4Ispp52EA1sOqzMz3Av,https://api.spotify.com/v1/tracks/5VA4Ispp52EA...,https://api.spotify.com/v1/audio-analysis/5VA4...,295920,4,low,Neutral,0.836536
27998,I Wish You Would,Taylor Swift,spotify:artist:06HL4z0CvFAxyc27GXpf02,['pop'],100,0.653,0.893,0,-5.966,1,...,audio_features,5gRYrtvyVyaCRvLt56OfuV,spotify:track:5gRYrtvyVyaCRvLt56OfuV,https://api.spotify.com/v1/tracks/5gRYrtvyVyaC...,https://api.spotify.com/v1/audio-analysis/5gRY...,207440,4,low,Neutral,0.834657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29380,My Favorite Moment of the Bee 3/Second,Original Broadway Cast Recording,spotify:artist:43qpnCSCXefs3GzxOrWzdz,['broadway'],3,0.353,0.275,4,-9.888,0,...,audio_features,1MsxKjQ5moDDHVBGeHEUwo,spotify:track:1MsxKjQ5moDDHVBGeHEUwo,https://api.spotify.com/v1/tracks/1MsxKjQ5moDD...,https://api.spotify.com/v1/audio-analysis/1Msx...,395027,4,high,Positive,0.165281
12770,Try - Live In Concert,John Mayer Trio,spotify:artist:68xuC1qXRhCUvGd4SSM655,['modern blues'],0,0.405,0.911,7,-6.842,0,...,audio_features,4OZtDR79Lwz1DblkJffzf3,spotify:track:4OZtDR79Lwz1DblkJffzf3,https://api.spotify.com/v1/tracks/4OZtDR79Lwz1...,https://api.spotify.com/v1/audio-analysis/4OZt...,413293,4,high,Positive,0.165064
29467,Angry Inch,Hedwig & The Angry Inch,spotify:artist:6NxcE1vZdlUZCnuitqIuHS,['show tunes'],0,0.498,0.873,8,-5.389,0,...,audio_features,7b5SsKiVaGArlVWRCYP6KL,spotify:track:7b5SsKiVaGArlVWRCYP6KL,https://api.spotify.com/v1/tracks/7b5SsKiVaGAr...,https://api.spotify.com/v1/audio-analysis/7b5S...,183293,4,high,Negative,0.161130
29472,Exquisite Corpse,Hedwig & The Angry Inch,spotify:artist:6NxcE1vZdlUZCnuitqIuHS,['show tunes'],0,0.209,0.929,11,-5.168,0,...,audio_features,6NSWY89cVbiNMdrd2mkjiw,spotify:track:6NSWY89cVbiNMdrd2mkjiw,https://api.spotify.com/v1/tracks/6NSWY89cVbiN...,https://api.spotify.com/v1/audio-analysis/6NSW...,183600,4,high,Positive,0.159461


In [86]:
def return_to_features(input_pid,num):
  rec_song_info = []
  recommended = recommend_playlist(input_pid).head(num) #30 x 26
  samples = generate_nonplaylist_features(input_pid) #m x n
  for song_index in recommended.index:
    rec_song_info.append(samples.loc[song_index])
  return rec_song_info

rec_song_info = return_to_features(0,500)

<ipython-input-84-5334da316217>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonplaylist_df['similarity'] = cosine_similarity(nonplaylist_features.values, input_playlist_features.values.reshape(1, -1))[:,0]


In [87]:
import numpy as np
from sklearn.metrics import accuracy_score

def metrics(input_pid):
  original = input_playlist_features_vector(input_pid) #n x 1
  round_original = original.apply(round_ht)
  rec_song_info_df = pd.DataFrame(rec_song_info)

  r_precision_list = []
  ndcg_list = []
  for index, song in rec_song_info_df.iterrows():
    r_precision_list.append(r_precision(song,round_original))
    ndcg_list.append(ndcg(song,round_original))
  r_precision_val =  sum(r_precision_list)/len(r_precision_list)
  ndcg_val =  sum(ndcg_list)/len(ndcg_list)
  print('r-precision value: ', r_precision_val, '\nndcg value: ', ndcg_val)



def r_precision(prediction, truth):
    prediction_set = set(prediction)
    truth_set = set(truth)
    intersect = prediction_set.intersection(truth_set)
    return float(len(intersect)) / len(truth_set)

def ndcg(predictions,truth):
    predictions_list = list(predictions)
    truth_list = list(truth)

    # Produces an ordered vector of 1.0 and 0.0
    score = [float(element in truth_list) for element in predictions_list]
    dcg  = np.sum(score / np.log2(1 + np.arange(1, len(score) + 1)))

    ones = np.ones([1,len(truth_list)])
    idcg = np.sum(ones / np.log2(1 + np.arange(1, len(truth_list) + 1)))

    return (dcg / idcg)


def round_ht(num):
  return round(num,100)

metrics(0)

r-precision value:  0.016159090909091033 
ndcg value:  0.9912984138592889


# Things to do next
1. Generate the accuracy of the recommended playlist
2. Expand to entire dataset.
3. Implement an input-playlist function.
4. Front end?